## Introducing the ArcGIS Python API

The ArcGIS Python API provides a way to explore and analyze spatial data in a more flexible way than ArcPy and with a heavy focus on web GIS.


### 01 - Import arcgis

We start by importing the <a href="https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html#gis" target="_blank">GIS</a> module, which is an object that will represent a single ArcGIS Online, ArcGIS Portal or ArcGIS Enterprise environment.

In [ ]:
from arcgis.gis import GIS
from IPython.display import display

### 02 - Login to ArcGIS Online

Since we're in a shared environment, we'll use a free account previously setup for this purpose.

In the following prompts, enter the username and password of your own ArcGIS Online account or the shared account made for this event. We use the `getpass` module to demonstrate obscuring passwords, which is helpful when sharing and publishing notebooks.

To confirm that we've successfully authenticated, we'll ask the API to return the logged in user's full name property.

In [ ]:
from getpass import getpass


username = input('Enter your ArcGIS Online username: ')
password = getpass('Enter your ArcGIS Online password: ')

try:
    gis = GIS('https://www.arcgis.com', username, password)
    full_name = gis.properties.user['fullName']
    print('Successfully authenticated: {} is logged in'.format(full_name)) 
except RuntimeError:
    print('Unable to authenticate')

### 03 - Features Module

The <a href="https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#featurelayer" target="_blank">FeatureLayer</a> is the primary concept for working with features in a GIS.

Users create, import, export, analyze, edit, and visualize features, i.e. entities in space as feature layers.

Let's take a publicly available ArcGIS Server layer and visualize, query and extract it in our notebook. We'll use a layer produced by TNRIS that represents low water crossings across Texas, inspect it for some important properties, view it on our web map and then export to a feature class.

In [ ]:
from arcgis.features import FeatureLayer

# An ArcGIS Server layer produced by TNRIS representing low water crossings across the state
tnris_layer_url = 'https://webservices.tnris.org/arcgis/rest/services/Low_Water_Crossings/Low_Water_Crossings/MapServer/0'
feature_layer = FeatureLayer(tnris_layer_url)


#### Querying the FeatureLayer

The ArcGIS Python API makes it easy to investigate and query our spatial data. Lets get a total record count for our feature layer and some other information like geometry type and spatial reference, then query the data down to Travis county.

Notice we need to escape the quotes around the query term Travis. Using escape sequences in strings tells Python to treat the following character as just another member of the string. We could have also provided double quotes. 

In [ ]:
record_count = feature_layer.query(where='1=1', return_count_only=True)


print('Feature geometry type: {}'.format(feature_layer.properties['geometryType']))
print('Feature spatial reference: {}'.format(str(feature_layer.properties['sourceSpatialReference']['latestWkid'])))
print('Feature record count: {}'.format(str(record_count)))


travis_county_crossings = feature_layer.query(where='COUNTY = \'TRAVIS\'')
travis_county_crossings.df

#### Visualize the FeatureLayer on a Map

Next, we create a new `map` instance and add our feature layer to it so we can visualize and interact with the data.

In [ ]:
map = gis.map('Austin, Texas', zoomlevel=12)
map.add_layer(travis_county_crossings)
map

#### Creating a SpatialDataFrame

In [ ]:
from arcgis.features import SpatialDataFrame

spatial_df = SpatialDataFrame.from_layer(feature_layer)
spatial_df


#### Exporting to a Feature Class

Let's export our SpatialDataFrame to a feature class so we can download for further analysis or for sending to co-workers. In this next cell, we're using the `to_featureclass` method of the SpatialDataFrame to easily write our features to a local file.

There's a lot happening here so don't worry if it looks a little confusing. Much of what we're doing is meant to be nice to the shared environment (writing intermediate files to a temporary directory, working inside <a href="https://en.wikibooks.org/wiki/Python_Programming/Context_Managers" target="_blank">context managers</a> to automatically clean up after ourselves, etc.)

What we'll end up with is a zip file in our home directory containing a shapefile of our SpatialDataFrame. We can then easily download it from the Jupyter Home page.

In [ ]:
import os
import zipfile
import tempfile

# Make sure the directory 'Slice-of-py' exists, and if it doesn't, create it.
output_path = os.path.join(os.path.expanduser('~'), 'Slice-of-py')
if not os.path.exists(output_path):
    os.mkdir(output_path)


zipname = input('Give your zip file a name: ')
    
    
# Create a new temporary directory that is guaranteed to be removed when we're done
with tempfile.TemporaryDirectory() as tempdir:
    
    # Write the shapefile to the temporary directory, giving it the name of our FeatureLayer
    spatial_df.to_featureclass(out_location=tempdir, out_name=feature_layer.properties.name)
    
    # Create a new zip file in our "home" directory using compression
    with zipfile.ZipFile(os.path.join(output_path, zipname + '.zip'), mode='w', compression=zipfile.ZIP_DEFLATED) as fc_zip:
        
        # Loop over the files we just created in the temporary directory
        for shpfile in os.listdir(tempdir):
            
            # Print and write the shapefile components to the zip file we just created
            print(shpfile)
            fc_zip.write(os.path.join(tempdir, shpfile), arcname=shpfile)

### 04 - Mapping Module

The mapping module allows us to interact with web maps and web scenes from our GIS. We can create new items or get references to and edit existing items.

Let's create a new web map and add our feature layer to it, then save to ArcGIS Online so we can share it with the world.

In [ ]:
from arcgis.mapping import WebMap
webmap = WebMap()
webmap.add_layer(feature_layer)

title = input('Give your WebMap a title: ')
item_properties = {
    'title': title,
    'snippet': 'Low Water Crossings in Texas provided by TNRIS',
    'tags': ['texas']
}

webmap.save(item_properties)